<a href="https://colab.research.google.com/github/Aye-Nyein-Thaw/TensorFlow-Beginner/blob/main/3_Feature_Extraction_Layer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Fine-Tuning

In [23]:
import tensorflow as tf
import tensorflow_datasets as tfds
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense, Conv2D, MaxPooling2D, Flatten

## Sequential Model

In [24]:
model = Sequential([
    Dense(6, name = 'Dense_0', input_shape = (4,)),
    Dense(4, name = 'Dense_1'),
    Dense(3, name = 'output')
])

model.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
Dense_0 (Dense)              (None, 6)                 30        
_________________________________________________________________
Dense_1 (Dense)              (None, 4)                 28        
_________________________________________________________________
output (Dense)               (None, 3)                 15        
Total params: 73
Trainable params: 73
Non-trainable params: 0
_________________________________________________________________


## Freeze 2nd layer

In [ ]:
model.layers[2].trainable = False
model.summary()

## Load the TensorFlow Hub Feature Vector

In [26]:
import tensorflow_hub as hub

handle_base = "mobilenet_v2"
pixels = 224
output_features = 1280

IMAGE_SIZE = (pixels, pixels)
MODULE_HANDLE ="https://tfhub.dev/google/tf2-preview/{}/feature_vector/4".format(handle_base)

# The output of the module is a batch of feature vectors. 
# For each input image, the feature vector has size num_features = 1280

## Feature extraction layer

In [27]:
feature_extractor = hub.KerasLayer(MODULE_HANDLE, input_shape=IMAGE_SIZE + (3,))

In [29]:
model = Sequential([
            feature_extractor,
            Dense(2, activation='softmax')
])

## Prepare Data

In [30]:
splits = ['train[:80%]', 'train[80%:90%]', 'train[90%:]']

splits, info = tfds.load('cats_vs_dogs', with_info=True, as_supervised=True, split=splits)

(train_examples, validation_examples, test_examples) = splits

In [31]:
num_examples = info.splits['train'].num_examples
num_classes = info.features['label'].num_classes

class_names = ['cat', 'dog']

def format_image(image, label):
    image = tf.image.resize(image, IMAGE_SIZE) / 255.0
    return  image, label
    
BATCH_SIZE =  32

train_batches = train_examples.shuffle(num_examples // 4).map(format_image).batch(BATCH_SIZE).prefetch(1)
validation_batches = validation_examples.map(format_image).batch(BATCH_SIZE).prefetch(1)
test_batches = test_examples.map(format_image).batch(BATCH_SIZE)

In [32]:
model.compile(optimizer = 'adam',
              loss = 'sparse_categorical_crossentropy',
              metrics = ['accuracy'])
history = model.fit(train_batches,
                    epochs=3,
                    validation_data=validation_batches)

Epoch 1/3
582/582 [==============================] - 31s 53ms/step - loss: 0.0572 - accuracy: 0.9811 - val_loss: 0.0382 - val_accuracy: 0.9867
Epoch 2/3
582/582 [==============================] - 30s 52ms/step - loss: 0.0299 - accuracy: 0.9893 - val_loss: 0.0370 - val_accuracy: 0.9867
Epoch 3/3
582/582 [==============================] - 30s 51ms/step - loss: 0.0250 - accuracy: 0.9916 - val_loss: 0.0404 - val_accuracy: 0.9888


In [33]:
import numpy as np

from tensorflow.keras.preprocessing.image import load_img, img_to_array

filepath = # your code here
test_img = load_img(filepath , target_size = (pixels, pixels))

# convert image to array
img_arr = img_to_array(test_img)

# normalize
img_arr = img_arr / 255.0

# expand_dimensions
img_arr = img_arr[np.newaxis, ...]
results = model.predict(img_arr)

prediction = class_names[np.argmax(results)]
print(prediction)

display(test_img)